## <span style="color:blue"> Python Exercise 11 </span>

## Accenni alle Reti Neurali 

### Neurone Artificiale

Modello matematico molto semplificato del neurone biologico. Ad ogni input $x_{i}$ è associato un peso $w_{i}$ con valore positivo o negativo per eccitare o inibire il neurone. 

#### Algoritmo del neurone

- 1 Caricare i valori degli input $x_{i}$ e dei pesi $w_{i}$
- 2 Calcolare la somma dei valori input pesata con i relativi pesi
- 3 Calcolare il valore della funzione di attivazione g con il risultato della somma pesata
- 4 L'output del neurone $y$ è il risultato della funzione di attivazione

$$y(x) = g\biggr(\displaystyle\sum_{i=1}^d w_{i}x_{i} + w_{0}\biggr)$$

#### Funzione di attivazione 

Determina la risposta del neurone

### Apprendimento della Rete

La programmazione serve solo per creare la rete e l'algoritmo di apprendimento (deve capire come comportarsi con l'input che riceve).

I pesi $w_{i}$, indispensabili da ottimizzare per ottenere un buon risultato, vengono scelti mediante le seguenti tecniche algoritmiche:

-  #### Funzione di costo .
    Rapprsenta la "perdita" in termini di efficacia predittiva, determinata dal modello rispetto ai valori reali che, in fase successiva, sono ad esso rapportati. Qusta funzione dipende dai parametri fondamentali (pesi) e tende a zero quanto più l'output del modello si avvicina ai valori di "testing". L'obbiettivo è quindi quello di minimizzare la funzione di costo.

- #### Optimizer. 
    Funzione che adatta i pesi, durante la procedura di training, per arrivare a minimizzare la funzione di costo. 


Noi ci occuperemo di reti ad *Apprendimento Supervisionato*. Vengo cioè presentati al computer degli input di esempio ed i relativi output desiderati, con lo scopo di apprendere una regola generale in grado di mappare gli input negli output.

Qui sotto verrà affrontato un problema di *regressione*. L'output ha un dominio continuo.

### Exercise 11.1

Esplorare come il codice sulla regressione lineare dipende dal numero di epoche, $N_{\mathrm{epochs}}$, dai numeri di punti, $N_{\mathrm{train}}$ e dal rumore $\sigma$. Migliorare il risultato operando su questi parametri.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import sin
import tensorflow as tf
from mpl_toolkits.mplot3d import Axes3D
from tensorflow import keras
from keras import optimizers, losses, metrics
from keras import activations
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from keras.layers import LeakyReLU
from tensorflow.keras import backend as K
from tensorflow.keras.utils import get_custom_objects

# target parameters of f(x) = m*x + b
m = 2 # slope
b = 1 # intersect

# generate training inputs
np.random.seed(0)
x_train = np.random.uniform(-1, 1, 500)
x_valid = np.random.uniform(-1, 1, 50)
x_valid.sort()
y_target = m * x_valid + b # ideal (target) linear function

sigma = 0.2 # noise standard deviation, for the moment it is absent
y_train = np.random.normal(m * x_train + b, sigma) # actual measures from which we want to guess regression parameters
y_valid = np.random.normal(m * x_valid + b, sigma)


plt.plot(x_valid, y_target)
plt.scatter(x_valid, y_valid, color='r')
plt.grid(True); plt.show()


model = tf.keras.Sequential()
model.add(Dense(1, input_shape=(1,)))

# compile the model choosing optimizer, loss and metrics objects
model.compile(optimizer='sgd', loss='mse', metrics=['mse'])


history = model.fit(x=x_train, y=y_train, 
          batch_size=32, epochs=50,
          shuffle=True, # a good idea is to shuffle input before at each epoch
          validation_data=(x_valid, y_valid))

score = model.evaluate(x_valid, y_valid, batch_size=32, verbose=1)

# print performance
print()
print('Test loss:', score[0])
print('Test accuracy:', score[1])

score = model.evaluate(x_valid, y_target, batch_size=32, verbose=1)

# print performance
print()
print('Test loss:', score[0])
print('Test accuracy:', score[1])

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='best')
plt.show()

### Exercise 11.2

Provare ad estendere il modello per un polinomio del terzo ordine:

$$
f(x)=4-3x-2x^2+3x^3
$$
for $x \in [-1,1]$.

Esplorare differenti valori per:

- the number of layers
- the number of neurons in each layer
- the activation function
- the optimizer
- the loss function

Giudicare i modelli NN vedendo quanto bene i fit predicono nuovi valori di test.


In [ ]:
d = 4
c = -3
b = -2
a = 3

np.random.seed(0)
x_train = np.random.uniform(-1, 1, 5000)
x_valid = np.random.uniform(-1, 1, 500)
x_valid.sort()
y_target = d + c * x_valid + b * x_valid*x_valid + a * x_valid * x_valid * x_valid

sigma = 0.3 ##noise
y_train = np.random.normal(d + c * x_train + b * x_train*x_train + a * x_train * x_train * x_train, sigma)
y_valid = np.random.normal(d + c * x_valid + b * x_valid*x_valid + a * x_valid * x_valid * x_valid, sigma)

plt.plot(x_valid,y_target)
plt.scatter(x_valid, y_valid, color = 'r')
plt.grid(True); #plt.show()

model = tf.keras.Sequential()
model.add(Dense(9, input_shape=(1,), activation='relu'))#numero di neuroni nel primo "hidden layer" e 1 parametro di input
model.add(Dense(18, activation='relu'))# dopo il primo layer non devo specificare più la size dell'input
model.add(Dense(27, activation='relu'))
model.add(Dense(1))#output

model.compile(optimizer='sgd', loss='mse', metrics=['mse'])

history = model.fit(x=x_train, y=y_train, batch_size=32, epochs=100, shuffle=True,validation_data=(x_valid,y_valid))

score = model.evaluate(x_valid, y_valid, batch_size=32,verbose=1)

print()
print('Test loss:', score[0])
print('Test Accuracy:',score[1])


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='best'); plt.show()


x_predicted = np.random.uniform(-1, 1, 200)
y_predicted = model.predict(x_predicted)
plt.scatter(x_predicted, y_predicted,color='r')
plt.plot(x_valid, y_target)
plt.grid(True); plt.show()


print("Input dataset shape: ", x_valid.shape)
# this model maps a 2 dims problem into an 9 dims
y_predicted = model.predict(x_valid, batch_size=128)
print("Predicted results shape: ", y_predicted.shape)
print(y_valid[4])
print(y_predicted[4])

### Analisi

I punti interni al data train si avvicinano bene al valore della funzione mentre quelli esterni si distaccano maggiornamente.

Le varie prove che sono state fatte vertevano sul variare numero di neuroni nei layers, variare numero di layers, variare la "action function" , l'optimizer, la "loss function", e le epoche. 

Ho visto che: 

- aumentare il numero di neuroni e di layer aiutava a fittare meglio la curva .
- La funzione di attivazione è ricaduta su quella più utilizzata in letteratura (relu, sia per hidden layers che per input layers).
- Ricordando che il "Gradient descend" è un processo iterativo, ho aumentato il numero di epohe aumentando così la bontà del mio fit (stando attenta a non aumentare troppo le opeche per evitare l'overfit). 


### Exercise 11.3

Provare a estendere il modello per fittare $f(x,y) = \sin(x^2+y^2)$ nel range $x \in [-3/2,3/2]$ and $y \in [-3/2,3/2]$.


In [ ]:
np.random.seed(0)

n_train = 5000
n_valid = 500

x_train = np.zeros((n_train,2)) #mi costruisco le matrici
y_train = np.zeros((n_train,2))

x_valid = np.zeros((n_valid,2))
y_valid = np.zeros((n_valid,2))

y_target = np.zeros(n_valid)
y_train = np.zeros(n_train)
y_valid = np.zeros(n_valid)

for i in range(n_train):
	x_train[i,0] = np.random.uniform(-1.5, 1.5)
	x_train[i,1] = np.random.uniform(-1.5, 1.5)

for i in range(n_valid):
	x_valid[i,0] = np.random.uniform(-1.5, 1.5)
	x_valid[i,1] = np.random.uniform(-1.5, 1.5)

for i in range(n_valid):
	y_target[i] = np.sin(x_valid[i,0]*x_valid[i,0] + x_valid[i,1]*x_valid[i,1])

sigma = 0.3 ##noise
for i in range(n_train):
	y_train[i] = np.random.normal(np.sin(x_train[i,0]*x_train[i,0] + x_train[i,1]*x_train[i,1]), sigma)

for i in range(n_valid):
	y_valid[i] = np.random.normal(np.sin(x_valid[i,0]*x_valid[i,0] + x_valid[i,1]*x_valid[i,1]), sigma)

    
fig = plt.figure()
ax = Axes3D(fig)
ax.plot_trisurf(x_valid[:,0],x_valid[:,1], y_valid,cmap='viridis', edgecolor = 'none')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.view_init(30,20)
plt.show()


model = tf.keras.Sequential()
model.add(Dense(12, input_shape=(2,), activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='relu'))

#relu = activations.relu(x_train,alpha=0.5)

model.compile(optimizer='sgd', loss='mse', metrics=['mse'])
#sgd = optimizers.SGD(lr=0.01, decay = 1e-6, momentum=0.9, nesterov=True)
#keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

history = model.fit(x=x_train, y=y_train, batch_size=50, epochs=100, shuffle=True,validation_data=(x_valid,y_valid))

score = model.evaluate(x_valid, y_valid, batch_size=50,verbose=1)

print()
print('Test loss:', score[0])
print('Test Accuracy:',score[1])


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='best'); plt.show()

x_predicted = np.zeros((500,2))

for i in range(500):
	x_predicted[i,0] = np.random.uniform(-1.5, 1.5)
	x_predicted[i,1] = np.random.uniform(-1.5, 1.5)

y_predicted = model.predict(x_predicted)

print (y_predicted[:,0].shape)

fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(x_predicted[:,0],x_predicted[:,1], y_predicted[:,0], c = y_predicted[:,0], marker = '.')
ax.plot_trisurf(x_valid[:,0],x_valid[:,1], y_target,cmap='viridis', color = 'none')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.view_init(30,20)
plt.show()


print("Input dataset shape: ", x_valid.shape)
# this model maps a 2 dims problem into a 1 dims
y_predicted = model.predict(x_valid, batch_size=128)
print("Predicted results shape: ", y_predicted.shape)
print(y_valid[2])
print(y_predicted[2])
